In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

# Nodes
delay = Delay(model, 'Delay')
fork1 = Fork(model, 'Fork1')
fork1.set_tasks_per_link(1)
fork11 = Fork(model, 'Fork1_1')
fork11.set_tasks_per_link(2)
join1 = Join(model, 'Join1', fork1)
join11 = Join(model, 'Join1_1', fork11)
queue1 = Queue(model, 'Queue1', SchedStrategy.PS)
queue2 = Queue(model, 'Queue2', SchedStrategy.PS)

# Classes
jobclass1 = ClosedClass(model, 'class1', 5, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 2, delay, 0)

# Service times
delay.set_service(jobclass1, Exp(0.25))
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(0.75))

delay.set_service(jobclass2, Exp(0.25))
queue1.set_service(jobclass2, Exp(2.0))
queue2.set_service(jobclass2, Exp(2.0))

In [3]:
# Routing
P = model.init_routing_matrix()

# Class 1 routing
P.set(jobclass1, jobclass1, delay, fork1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join1, 1.0)
P.set(jobclass1, jobclass1, queue2, join1, 1.0)
P.set(jobclass1, jobclass1, join1, delay, 1.0)

# Class 2 routing
P.set(jobclass2, jobclass2, delay, fork11, 1.0)
P.set(jobclass2, jobclass2, fork11, fork1, 1.0)
P.set(jobclass2, jobclass2, fork1, queue1, 1.0)
P.set(jobclass2, jobclass2, fork1, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join1, 1.0)
P.set(jobclass2, jobclass2, queue2, join1, 1.0)
P.set(jobclass2, jobclass2, join1, join11, 1.0)
P.set(jobclass2, jobclass2, join11, delay, 1.0)

model.link(P)

In [4]:
# Solve with different solverssolver = []# JMT solverjmt_options = SolverJMT.defaultOptions()jmt_options.seed = 23000solver.append(SolverJMT(model, jmt_options))# MVA solversolver.append(SolverMVA(model))# Get average tablesAvgTable = []for s in solver:    print(f'\nSOLVER: {s.get_name()}')    avg_table = s.get_avg_table()    AvgTable.append(avg_table)


SOLVER: SolverJMT
JMT Model: /tmp/workspace/jsim/10319964193960901386/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.942999s.
   Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0    Delay   class1  2.0048  2.0048  4.0521  4.0521  0.4957  0.4959
1    Delay   class2  1.0274  1.0274  3.9767  3.9767  0.2546  0.2546
2    Join1   class1  2.2220  0.0000  2.2680  2.2680  0.9824  0.4957
3    Join1   class2  1.0298  0.0000  0.9969  0.9969  1.0307  0.4962
5  Join1_1   class2  0.5092  0.0000  1.0146  1.0146  0.4962  0.2546
6   Queue1   class1  1.3422  0.4968  2.6209  1.3105  0.4959  0.4958
7   Queue1   class2  0.7907  0.2531  1.5417  0.7709  0.4975  0.5027
8   Queue2   class1  2.4293  0.6700  4.9108  2.4554  0.4959  0.4956
9   Queue2   class2  1.0851  0.2506  2.2152  1.1076  0.4975  0.4990

SOLVER: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.101014s.
   Station JobClass    QLen    Util   RespT  ResidT    ArvR